In [ ]:
!pip install easycolab
import easycolab as ec
ec.mount()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Opening directory /content/gdrive/My Drive/
Done!


In [ ]:
import numpy as np
from scipy.stats import spearmanr, kendalltau
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score

In [ ]:
import json, pickle

tournaments = pickle.load(open('chgk/tournaments.pkl', 'rb'))
results = pickle.load(open('chgk/results.pkl', 'rb'))
players = pickle.load(open('chgk/players.pkl', 'rb'))

# Filtering

In [ ]:
tour_ids = list(tournaments.keys())

In [ ]:
import operator

data_2019 = {}
for idx in tour_ids:
    if (tournaments[idx]['dateStart'][:4] == '2019'):
        if len(results[idx]) != 0:
            
            mask_len = 0
            for i, item in enumerate(results[idx]):
                if not ((not 'mask' in item.keys()) or (item['mask'] == None) or (len(item['teamMembers']) == 0)):
                    if len(item['mask']) > mask_len:
                        mask_len = len(item['mask'])
            
            for i, item in enumerate(results[idx]):
                teams = {}
                pl_ids = []
                cnt = 0 
                if not ((not 'mask' in item.keys()) or (item['mask'] == None) or (len(item['teamMembers']) == 0)):
                    #if not (('X' in item['mask']) or ('?' in item['mask'])):
                        
                    if mask_len == len(item['mask']):
                        for player in item['teamMembers']:
                            if not ((not 'player' in player) or (not 'id' in player['player'])):
                                pl_ids.append(player['player']['id'])
                            teams[item['team']['id']] = (item['mask'], pl_ids)
                
                if len(teams) != 0:
                    if not idx in data_2019:
                        data_2019[idx] = []
                    data_2019[idx].append(teams)

In [ ]:
ids_2019 = list(data_2019.keys())

In [ ]:
player_id_2019 = []
player_mask_2019 = []
q_id_2019 = []
players_2019 = {}

for idx in ids_2019:
    for item in data_2019[idx]:
        mask = list(item.values())[0][0]
        players_ = list(item.values())[0][1]
        for player in players_:
            if not player in players_2019:
                players_2019[player] = 1
            for i, q in enumerate(mask):
                if q != 'X' and q != '?':
                    player_id_2019.append(player)
                    q_id_2019.append(str(idx) + '_' + str(i))
                    player_mask_2019.append(int(q))

In [ ]:
team_2019 = []
team_mask_2019 = []
for idx in ids_2019:
    for item in data_2019[idx]:
        team = list(item.keys())[0]
        mask = list(item.values())[0][0]
        for i, q in enumerate(mask):
            if q != 'X' and q != '?':
                team_2019.append(team)
                team_mask_2019.append(int(q))
                #q_id_2019.append(str(idx) + '_' + str(i))
                #player_mask_2019.append(int(q))

In [ ]:
team_dict_2019 = {}
for idx in ids_2019:
    for item in data_2019[idx]:
        team = list(item.keys())[0]
        players_ = list(item.values())[0][1]
        team_dict_2019[team] = players_

In [ ]:
len(team_dict_2019.keys())

11420

In [ ]:
len(team_mask_2019)

3303169

In [ ]:
import operator

data_2020 = {}

for idx in tour_ids:
    if (tournaments[idx]['dateStart'][:4] == '2020'):
        if len(results[idx]) != 0:
            teams = {}

            for i, item in enumerate(results[idx]):     
                pl_ids = []
                for player in item['teamMembers']:
                    if not ((not 'player' in player) or (not 'id' in player['player']) or (not player['player']['id'] in players_2019.keys())):
                        pl_ids.append(player['player']['id'])
                    teams[item['team']['id']] = pl_ids
                
            if len(teams) > 1:
                if not idx in data_2020:
                    data_2020[idx] = []
                data_2020[idx].append(teams)

In [ ]:
ids_2020 = list(data_2020.keys())

In [ ]:
players_2020 = {}

for idx in ids_2020:
    for item in data_2020[idx]:
        team_keys = list(item.keys())
        for key in team_keys:
            players_ = item[key]
            for player in players_:
                if not player in players_2020:
                    players_2020[player] = 1

In [ ]:
player_id_2020 = list(players_2020.keys())

In [ ]:

player_id = np.array(player_id_2019 + player_id_2020)
player_id = player_id.reshape(-1, 1)

q_id_2019 = np.array(q_id_2019)
q_id_2019 = q_id_2019.reshape(-1, 1)

In [ ]:
enc_1 = OneHotEncoder()
X_train_player_id = enc_1.fit_transform(player_id)

In [ ]:
X_train_1 = X_train_player_id[:len(player_id_2019)]
X_test_1 = X_train_player_id[len(player_id_2019):]

In [ ]:
enc_2 = OneHotEncoder()
X_train_2 = enc_2.fit_transform(q_id_2019)

In [ ]:
X_train = hstack([X_train_1, X_train_2])

In [ ]:
y_train = np.array(player_mask_2019)

In [ ]:
X_train.shape

(17753259, 90727)

In [ ]:
len(player_mask_2019)

17753259

# Baseline model

In [ ]:
#from sklearn.linear_model import LogisticRegression
#model = LogisticRegression(solver = 'liblinear').fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import Ridge
y_train_ = np.clip(y_train, 1e-8, 1 - 1e-8)
y_train_ = np.log(y_train_ / (1 - y_train_))
model = Ridge(alpha=0.1)
model.fit(X_train, y_train_)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

# Baseline team rating

In [ ]:
p = 1/(1 + np.exp(- X_test_1 @ model.coef_[:X_test_1.shape[1]] - model.intercept_))

In [ ]:
player_2020_rate = {}
for i in range(len(player_id_2020)):
    player_2020_rate[player_id_2020[i]] = p[i]

In [ ]:
player_group_id_2020 = []
team_id_group_2020 = []
for idx in ids_2020:
    team_id_group_2020.append([])
    player_group_id_2020.append([])
    items = list(data_2020[idx][0].keys())
    for i, item in enumerate(results[idx]):
        if item['team']['id'] in items:
            team_id_group_2020[-1].append(item['team']['id'])
            player_group_id_2020[-1].append([])
            for player in item['teamMembers']:
                if not ((not 'player' in player) or (not 'id' in player['player']) or (not player['player']['id'] in players_2019.keys())):
                    player_group_id_2020[-1][-1].append(player['player']['id'])

In [ ]:
team_rating_2020 = []
pos = []

corr_sp = []
corr_k = []
for i in range(len(team_id_group_2020)):
    team_rating_2020.append([])
    pos.append([])
    for j in range(len(team_id_group_2020[i])):
        rating = 0
        for k in range(len(player_group_id_2020[i][j])):
            #if player_2020_rate[player_group_id_2020[i][j][k]] > rating:
            rating += player_2020_rate[player_group_id_2020[i][j][k]]
        team_rating_2020[i].append(rating/max(1, len(player_group_id_2020[i][j])))
        pos[i].append(j)
    corr_sp.append(-spearmanr(pos[i], team_rating_2020[i])[0])
    corr_k.append(-kendalltau(pos[i], team_rating_2020[i])[0])

## Spearman

In [ ]:
np.array(corr_sp).mean(0)

0.7390809567759947

## Kendall tau

In [ ]:
np.array(corr_k).mean(0)

0.5755465835987852

# EM-algorithm

In [ ]:
def sigmoid(x, bias0=None, bias=None):
    if (bias == None) and (bias0 == None):
        return 1/(1 + np.exp(- x))
    elif (bias != None) and (bias0 == None):
        return 1/(1 + np.exp(- x - bias))
    elif (bias == None) and (bias0 != None):
        return 1/(1 + np.exp(- x - bias0))
    elif (bias != None) and (bias0 != None):
        return 1/(1 + np.exp(- x - bias + bias0))

In [ ]:
def e_step(X, y, theta, bias0=None, bias=None):
    z = sigmoid(X @ theta, bias0=bias0, bias=bias)
    z[y == 0.] = 0.
    return z

In [ ]:
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

def m_step(X, z):
    z = np.clip(z, 1e-8, 1 - 1e-8)
    inv_sig_z = np.log(z / (1 - z))
    lr = Ridge(alpha=0.1)
    lr.fit(X, inv_sig_z)
    theta = lr.coef_
    bias = lr.intercept_
    return theta, bias

In [ ]:
def calculate_score(p2):
    player_2020_rate2 = {}
    for i in range(len(player_id_2020)):
        player_2020_rate2[player_id_2020[i]] = p2[i]
    team_rating_2020 = []
    pos = []
    corr_sp2 = []
    corr_k2 = []
    for i in range(len(team_id_group_2020)):
        team_rating_2020.append([])
        pos.append([])
        for j in range(len(team_id_group_2020[i])):
            rating = 0
            for k in range(len(player_group_id_2020[i][j])):
                rating += player_2020_rate2[player_group_id_2020[i][j][k]]
            team_rating_2020[i].append(rating/max(1, len(player_group_id_2020[i][j])))
            pos[i].append(j)
        corr_sp2.append(-spearmanr(pos[i], team_rating_2020[i])[0])
        corr_k2.append(-kendalltau(pos[i], team_rating_2020[i])[0])
    return np.array(corr_sp2).mean(0), np.array(corr_k2).mean(0)

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

theta = model.coef_
bias0 = model.intercept_
bias = None
sp_ = []
ke_ = []
for epoch in range(5):
    z = e_step(X_train, y_train, theta, bias0=bias0, bias=bias)
    theta, bias = m_step(X_train, z)
    p2 = sigmoid(X_test_1 @ theta[:X_test_1.shape[1]], bias=bias)
    sp, ke = calculate_score(p2)
    print('Spearman: %.8f'%sp + ' Kendall: %.8f'%ke)

Spearman: 0.73854006 Kendall: 0.57627997
Spearman: 0.74058913 Kendall: 0.57828338
Spearman: 0.74033247 Kendall: 0.57837626
Spearman: 0.74087319 Kendall: 0.57937715
Spearman: 0.74096001 Kendall: 0.57946648


# Competition rating

## Baseline

In [ ]:
q_p = 1/(1 + np.exp(- X_train_2 @ model.coef_[X_test_1.shape[1]:]))

In [ ]:
tournament_questions = {}
for i in range(len(q_id_2019)):
    question = q_id_2019[i][0].split('_')
    if not question[0] in tournament_questions.keys():
        tournament_questions[question[0]] = [q_p[i]]
    else:
        tournament_questions[question[0]].append(q_p[i])

In [ ]:
for key in tournament_questions.keys():
    tournament_questions[key] = np.array(tournament_questions[key]).mean()

In [ ]:
tournament_questions_sorted = sorted(tournament_questions.items(), key=lambda kv: kv[1])
print('First 10')
for i in range(10):
    print(tournaments[int(tournament_questions_sorted[i][0])]['name'])
print()
print('Last 10')
for i in range(len(tournament_questions_sorted) - 10, len(tournament_questions_sorted)):
    print(tournaments[int(tournament_questions_sorted[i][0])]['name'])

First 10
Чемпионат Санкт-Петербурга. Первая лига
Угрюмый Ёрш
Синхрон высшей лиги Москвы
Чемпионат Мира. Финал. Группа С
Записки охотника
Чемпионат Мира. Этап 3. Группа С
Чемпионат Мира. Этап 3. Группа В
Первенство правого полушария
Memento memes IV
Ускользающая сова

Last 10
(а)Синхрон-lite. Лига старта. Эпизод V
Школьный Синхрон-lite. Выпуск 2.5
Открытый кубок УМЭД
Школьный Синхрон-lite. Выпуск 3.1
Школьный Синхрон-lite. Выпуск 3.4
Школьный Синхрон-lite. Выпуск 3.3
Школьная лига
Школьная лига. II тур.
Школьная лига. III тур.
Школьная лига. I тур.


## EM

In [ ]:
q_p = 1/(1 + np.exp(- X_train_2 @ theta[X_test_1.shape[1]:]))
tournament_questions = {}
for i in range(len(q_id_2019)):
    question = q_id_2019[i][0].split('_')
    if not question[0] in tournament_questions.keys():
        tournament_questions[question[0]] = [q_p[i]]
    else:
        tournament_questions[question[0]].append(q_p[i])
for key in tournament_questions.keys():
    tournament_questions[key] = np.array(tournament_questions[key]).mean()
tournament_questions_sorted = sorted(tournament_questions.items(), key=lambda kv: kv[1])
print('First 10')
for i in range(10):
    print(tournaments[int(tournament_questions_sorted[i][0])]['name'])
print()
print('Last 10')
for i in range(len(tournament_questions_sorted) - 10, len(tournament_questions_sorted)):
    print(tournaments[int(tournament_questions_sorted[i][0])]['name'])

First 10
Чемпионат Санкт-Петербурга. Первая лига
Синхрон высшей лиги Москвы
Чемпионат Мира. Финал. Группа С
Угрюмый Ёрш
Чемпионат Мира. Этап 3. Группа С
Записки охотника
Первенство правого полушария
День D
Чемпионат Мира. Этап 3. Группа В
Memento memes IV

Last 10
Открытый кубок УМЭД
Кубок Караганды
Школьный синхрон-lite. Сезон 3
Школьный Синхрон-lite. Выпуск 3.4
Школьный Синхрон-lite. Выпуск 3.1
Школьный Синхрон-lite. Выпуск 3.3
Школьная лига. II тур.
Школьная лига
Школьная лига. III тур.
Школьная лига. I тур.


# Players rating:

## Baseline

In [ ]:
player_2020_rate_sorted = sorted(player_2020_rate.items(), key=lambda kv: kv[1], reverse=True)
for i in range(50):
    cnt = 0
    for j in range(len(player_id_2019)):
        if int(player_2020_rate_sorted[i][0]) == player_id_2019[j]:
            cnt += 1
    print(players[int(player_2020_rate_sorted[i][0])]['name'] + ' ' + players[int(player_2020_rate_sorted[i][0])]['surname'] + ' ' + str(cnt))
    


Максим Руссо 2178
Александра Брутер 2692
Иван Семушин 3774
Михаил Савченков 3215
Максим Пилипенко 36
Артём Сорожкин 4849
Сергей Спешков 3737
Станислав Мереминский 1584
Михаил Левандовский 1457
Ирина Прокофьева 1065
Илья Новиков 1589
Александр Мосягин 1112
Михаил Царёв 501
Игорь Мокин 1176
Александр Либер 3787
Дмитрий Карякин 1304
Мария Кленницкая 1172
Антон Саксонов 1194
Александр Марков 2903
Елена Александрова 48
Алексей Гилёв 4450
Евгений Спектор 266
Сусанна Бровер 600
Николай Крапиль 1652
Наталья Горелова 2093
Наталья Кудряшова 2626
Юрий Выменец 3306
Сергей Николенко 2219
Юлия Архангельская 1614
Михаил Новосёлов 2959
Михаил Матвеев 1025
Евгений Дёмин 537
Евгений Пашковский 1436
Владислав Порицкий 74
Антон Чернин 1496
Николай Коврижных 3056
Мария Юнгер 634
Андрей Островский 2612
Борис Моносов 1092
Александр Мартынов 1512
Александр Коробейников 1401
Иделия Айзятулова 2174
Наиль Фарукшин 3138
Александр Фингеров 1531
Владислава Плохих 39
Александр Людикайнен 295
Артём Митрофанов 757
Ели

## EM

In [ ]:
player_2020_rate2 = {}
for i in range(len(player_id_2020)):
    player_2020_rate2[player_id_2020[i]] = p2[i]
player_2020_rate_sorted2 = sorted(player_2020_rate2.items(), key=lambda kv: kv[1], reverse=True)

for i in range(50):
    cnt = 0
    for j in range(len(player_id_2019)):
        if int(player_2020_rate_sorted2[i][0]) == player_id_2019[j]:
            cnt += 1
    print(players[int(player_2020_rate_sorted2[i][0])]['name'] + ' ' + players[int(player_2020_rate_sorted2[i][0])]['surname'] + ' ' + str(cnt))

Максим Руссо 2178
Максим Пилипенко 36
Александра Брутер 2692
Иван Семушин 3774
Михаил Савченков 3215
Артём Сорожкин 4849
Сергей Спешков 3737
Станислав Мереминский 1584
Михаил Левандовский 1457
Ирина Прокофьева 1065
Александр Мосягин 1112
Алексей Гилёв 4450
Илья Новиков 1589
Сусанна Бровер 600
Антон Саксонов 1194
Игорь Мокин 1176
Владислава Плохих 39
Александр Марков 2903
Александр Либер 3787
Наталья Горелова 2093
Наталья Кудряшова 2626
Николай Крапиль 1652
Андрей Островский 2612
Юлия Архангельская 1614
Евгений Спектор 266
Юрий Выменец 3306
Михаил Матвеев 1025
Дмитрий Карякин 1304
Мария Кленницкая 1172
Михаил Новосёлов 2959
Михаил Царёв 501
Евгений Пашковский 1436
Николай Коврижных 3056
Антон Чернин 1496
Мария Юнгер 634
Александр Мартынов 1512
Борис Моносов 1092
Светлана Иванцова 817
Александр Коробейников 1401
Денис Пахомов 69
Кирилл Михайлов 729
Александр Фингеров 1531
Сергей Николенко 2219
Алексей Белоглазов 862
Наиль Фарукшин 3138
Александр Людикайнен 295
Елизавета Овдеенко 2292
Евг